<a href="https://colab.research.google.com/github/surya323-ma/EV_Adoption_Forecasting/blob/main/EV_Adoption_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Import Required Libraries

In [ ]:
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
from google.colab import files
uploaded=files.upload()
import pandas as pd
file_name = next(iter(uploaded)) # Get the name of the uploaded file
df=pd.read_csv(file_name)
print(df.columns)

df.head(10)
df.shape

Saving Electric_Vehicle.csv to Electric_Vehicle.csv
Index(['Date', 'County', 'State', 'Vehicle Primary Use',
       'Battery Electric Vehicles (BEVs)',
       'Plug-In Hybrid Electric Vehicles (PHEVs)',
       'Electric Vehicle (EV) Total', 'Non-Electric Vehicle Total',
       'Total Vehicles', 'Percent Electric Vehicles'],
      dtype='object')


(20819, 10)

Explore and Understand the Data

In [11]:
df.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,September 30 2022,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,December 31 2022,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,January 31 2020,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,June 30 2022,Ferry,WA,Truck,0,0,0,"3,575","3,575",0.00
4,July 31 2021,Douglas,CO,Passenger,0,1,1,83,84,1.19


In [12]:
df.shape

(20819, 10)

In [13]:
df.info

<bound method DataFrame.info of                     Date          County State Vehicle Primary Use  \
0      September 30 2022       Riverside    CA           Passenger   
1       December 31 2022  Prince William    VA           Passenger   
2        January 31 2020          Dakota    MN           Passenger   
3           June 30 2022           Ferry    WA               Truck   
4           July 31 2021         Douglas    CO           Passenger   
...                  ...             ...   ...                 ...   
20814    January 31 2023      Rockingham    NH           Passenger   
20815       July 31 2020     Carson City    NV           Passenger   
20816   February 28 2022          Island    WA           Passenger   
20817   December 31 2020       San Diego    CA           Passenger   
20818   November 30 2019       Goochland    VA           Passenger   

      Battery Electric Vehicles (BEVs)  \
0                                    7   
1                                    1   
2                                    0   
3                                    0   
4                                    0   
...                                ...   
20814                                1   
20815                                1   
20816                              744   
20817                               14   
20818                                3   

      Plug-In Hybrid Electric Vehicles (PHEVs) Electric Vehicle (EV) Total  \
0                                            0                           7   
1                                            2                           3   
2                                            1                           1   
3                                            0                           0   
4                                            1                           1   
...                                        ...                         ...   
20814                                        0                           1   
20815                                        0                           1   
20816                                      350                       1,094   
20817                                        2                          16   
20818                                        1                           4   

      Non-Electric Vehicle Total Total Vehicles  Percent Electric Vehicles  
0                            460            467                       1.50  
1                            188            191                       1.57  
2                             32             33                       3.03  
3                          3,575          3,575                       0.00  
4                             83             84                       1.19  
...                          ...            ...                        ...  
20814                         14             15                       6.67  
20815                         10             11                       9.09  
20816                     62,257         63,351                       1.73  
20817                      2,724          2,740                       0.58  
20818                        271            275                       1.45  

[20819 rows x 10 columns]>

In [14]:

df.isnull().sum()

,0
Date,0
County,86
State,86
Vehicle Primary Use,0
Battery Electric Vehicles (BEVs),0
Plug-In Hybrid Electric Vehicles (PHEVs),0
Electric Vehicle (EV) Total,0
Non-Electric Vehicle Total,0
Total Vehicles,0
Percent Electric Vehicles,0


Check if any column contain outliers.

In [15]:
# Compute Q1 and Q3
Q1 = df['Percent Electric Vehicles'].quantile(0.25)
Q3 = df['Percent Electric Vehicles'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print('lower_bound:', lower_bound)
print('upper_bound:', upper_bound)

# Identify outliers
outliers = df[(df['Percent Electric Vehicles'] < lower_bound) | (df['Percent Electric Vehicles'] > upper_bound)]
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])


lower_bound: -3.5174999999999996
upper_bound: 6.9025
Number of outliers in 'Percent Electric Vehicles': 2476


In [16]:
# Converts the "Date" column to actual datetime objects
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Removes rows where "Date" conversion failed
df = df[df['Date'].notnull()]
df = df[df['Electric Vehicle (EV) Total'].notnull()]

df['County'] = df['County'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')

print("Missing after fill:")
print(df[['County', 'State']].isnull().sum())
df.head()

Missing after fill:
County    0
State     0
dtype: int64


,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,2022-09-30,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,2022-12-31,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,2020-01-31,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,2022-06-30,Ferry,WA,Truck,0,0,0,"3,575","3,575",0.00
4,2021-07-31,Douglas,CO,Passenger,0,1,1,83,84,1.19


In [17]:
# Cap the outliers - it keeps all the data while reducing the skew from extreme values.

df['Percent Electric Vehicles'] = np.where(df['Percent Electric Vehicles'] > upper_bound, upper_bound,
                                 np.where(df['Percent Electric Vehicles'] < lower_bound, lower_bound, df['Percent Electric Vehicles']))

# Identify outliers
outliers = df[(df['Percent Electric Vehicles'] < lower_bound) | (df['Percent Electric Vehicles'] > upper_bound)]
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])

Number of outliers in 'Percent Electric Vehicles': 0


# New Section